In [1]:
import pandas as pd
import csv
import matplotlib as plot

In [2]:
deadout_df = pd.read_csv("bee_data/bee_loss_deadout.csv")
collapse_df = pd.read_csv("bee_data/colony_collapse_bees.csv")

deadout_df.dropna(axis=1, how="all").copy()
collapse_df.dropna(axis=1, how="all").copy()

,Program,Year,Period,Geo Level,State,State ANSI,watershed_code,Commodity,Data Item,Domain,Domain Category,Value
0,SURVEY,2020,JAN THRU MAR,NATIONAL,US TOTAL,NaN,0,HONEY,"HONEY, BEE COLONIES - LOSS, COLONY COLLAPSE DI...",TOTAL,NOT SPECIFIED,"105,240"
1,SURVEY,2020,JAN THRU MAR,STATE,ALABAMA,1.0,0,HONEY,"HONEY, BEE COLONIES - LOSS, COLONY COLLAPSE DI...",TOTAL,NOT SPECIFIED,70
2,SURVEY,2020,JAN THRU MAR,STATE,ARIZONA,4.0,0,HONEY,"HONEY, BEE COLONIES - LOSS, COLONY COLLAPSE DI...",TOTAL,NOT SPECIFIED,"4,300"
3,SURVEY,2020,JAN THRU MAR,STATE,ARKANSAS,5.0,0,HONEY,"HONEY, BEE COLONIES - LOSS, COLONY COLLAPSE DI...",TOTAL,NOT SPECIFIED,390
4,SURVEY,2020,JAN THRU MAR,STATE,CALIFORNIA,6.0,0,HONEY,"HONEY, BEE COLONIES - LOSS, COLONY COLLAPSE DI...",TOTAL,NOT SPECIFIED,"19,000"
...,...,...,...,...,...,...,...,...,...,...,...,...
247,SURVEY,2015,YEAR,NATIONAL,US TOTAL,NaN,0,HONEY,"HONEY, BEE COLONIES - LOSS, COLONY COLLAPSE DI...",INVENTORY OF BEE COLONIES,INVENTORY OF BEE COLONIES: (LESS THAN 5 COLONIES),"7,000"
248,SURVEY,2015,JAN THRU MAR,NATIONAL,US TOTAL,NaN,0,HONEY,"HONEY, BEE COLONIES - LOSS, COLONY COLLAPSE DI...",TOTAL,NOT SPECIFIED,"92,250"
249,SURVEY,2015,APR THRU JUN,NATIONAL,US TOTAL,NaN,0,HONEY,"HONEY, BEE COLONIES - LOSS, COLONY COLLAPSE DI...",TOTAL,NOT SPECIFIED,"56,760"
250,SURVEY,2015,JUL THRU SEP,NATIONAL,US TOTAL,NaN,0,HONEY,"HONEY, BEE COLONIES - LOSS, COLONY COLLAPSE DI...",TOTAL,NOT SPECIFIED,"88,390"


In [3]:
deadout_df["Value"] = deadout_df["Value"].str.replace(",","")
deadout_df['Value'] = deadout_df['Value'].astype('int64')
deadout = deadout_df.groupby(['Year', 'State',], as_index=False)[['Value']].sum()
deadout = deadout.loc[deadout['Year'] >= 2019]
deadout = deadout.rename(columns = {
    'Year': 'year',
    'State': 'state',
    'Value': 'deadout'
    })
deadout

,year,state,deadout
188,2019,ALABAMA,2300
189,2019,ARIZONA,11500
190,2019,ARKANSAS,10200
191,2019,CALIFORNIA,464000
192,2019,COLORADO,17820
...,...,...,...
277,2020,VIRGINIA,2300
278,2020,WASHINGTON,4600
279,2020,WEST VIRGINIA,1670
280,2020,WISCONSIN,3200


In [4]:
collapse_df["Value"] = collapse_df["Value"].str.replace(",","")
collapse_df['Value'] = collapse_df['Value'].astype('int64')
collapse = collapse_df.groupby(['Year', 'State',], as_index=False)[['Value']].sum()
collapse = collapse.loc[collapse['Year'] >= 2019]
collapse = collapse.rename(columns = {
    'Year': 'year',
    'State': 'state',
    'Value': 'cc_syn'
    })
collapse

,year,state,cc_syn
4,2019,ALABAMA,430
5,2019,ARIZONA,5670
6,2019,ARKANSAS,1870
7,2019,CALIFORNIA,44500
8,2019,COLORADO,4600
...,...,...,...
92,2020,VIRGINIA,540
93,2020,WASHINGTON,6870
94,2020,WEST VIRGINIA,330
95,2020,WISCONSIN,1690
